In [35]:
#returns a dataframe containing all info of all search results
#to get latitude, use get_search_location()["lat"][<index>]
#to get longitude, use get_search_location()["lng"][<index>]
def get_search_location(search_term):
    #This uses HERE api but seems not as good as geopy
    """import requests
    import pandas as pd
    
    URL = "https://geocode.search.hereapi.com/v1/geocode"
    #input("Enter the location here: ")
    api_key = "VhUOdWKYgNfrPSPdaYcBarB9OLFvT-rLAqW4wBc-Wy0" # Acquired from developer.here.com
    PARAMS = {'apikey':api_key,'q':str(search_term)} 

    # sending get request and saving the response as response object 
    response = requests.get(url = URL, params = PARAMS)
    location = pd.DataFrame(response.json())
    location = pd.DataFrame(location["items"].apply(pd.Series))
    
    return location"""
    
    #This uses Nominatim api through geopy
    from geopy.geocoders import Nominatim
    import pandas as pd
    
    geolocator = Nominatim(user_agent="user-636@project-306014.iam.gserviceaccount.com")
    search = geolocator.geocode
    
    location = search(search_term + ", Singapore", exactly_one = False) #exactly_one = False to return more than one result
    location = pd.DataFrame(location)
    location = pd.concat([location.iloc[:,:2], location[1].apply(pd.Series)], axis=1) #split lat and lng
    location.columns = ["address","coordinates", "lat", "lng"]
    
    return location

In [163]:
#returns a dataframe containing all info of user's IP location
#to get latitude, use get_user_location()["lat"]
#to get longotude, use get_user_location()["lng"]
def get_user_location():
    import geocoder
    import pandas as pd
    
    g=geocoder.ip("me")
    location = pd.json_normalize(g.json)
    return location

In [42]:
get_search_location("wild wild wet")

,address,coordinates,lat,lng
0,"Wild Wild Wet, Tampines Park Connector, Pasir ...","(1.3780454500000001, 103.9545492890241)",1.378045,103.954549


In [182]:
get_user_location()

,address,city,country,ip,lat,lng,ok,org,postal,state,status,raw.ip,raw.city,raw.region,raw.country,raw.loc,raw.org,raw.postal,raw.timezone,raw.readme
0,"Singapore, Singapore, SG",Singapore,SG,155.69.182.22,1.2897,103.8501,True,AS9419 Nanyang Technological University,048508,Singapore,OK,155.69.182.22,Singapore,Singapore,SG,"1.2897,103.8501",AS9419 Nanyang Technological University,048508,Asia/Singapore,https://ipinfo.io/missingauth


In [33]:
def carparks_nearby(df, coordinates, radius):
    from geopy.distance import distance
    import pandas as pd

    df["Location"] = df["Location"].str.split(" ", n=1, expand=False)
    df = pd.concat([df["CarParkID"], (pd.DataFrame(df["Location"].to_list(), columns=['lat', 'lng']))], axis=1)
    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['lng'] = pd.to_numeric(df['lng'], errors='coerce')

    radius_in_degree = radius/111.2
    df = pd.DataFrame(df[abs(df["lat"]-coordinates[0]) < radius_in_degree])
    df = pd.DataFrame(df[abs(df["lng"]-coordinates[1]) < radius_in_degree])
    return list(set(df["CarParkID"]))

In [ ]:
#Code to display map in jupyter notebook / python